In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from tensorflow import keras
from keras.utils.np_utils import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
# from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda

from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
# from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
from keras.optimizers import *
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth= True
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
random_seed = 
np.random.seed(random_seed)  
tf.set_random_seed(random_seed)
KTF.set_session(session)

In [ ]:
from Hypers import *
from Utils import *
from Preprocessing import *
from Generator import *
from models import *

In [ ]:
data_root_path = ''
embedding_path = ''
KG_root_path = ''

In [ ]:
news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict = read_news(data_root_path)
news_title,news_vert,news_subvert,news_entity,news_content=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict)

In [ ]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path, word_dict)
content_word_embedding_matrix, have_word = load_matrix(embedding_path, content_dict)

In [ ]:
graph, EntityId2Index, EntityIndex2Id, entity_embedding = load_entity_metadata(KG_root_path)
news_entity_KG = load_news_entity(news,EntityId2Index,data_root_path)

In [ ]:
news_entity_index = parse_zero_hop_entity(EntityId2Index,news_entity_KG,news_index,max_entity_num)
one_hop_entity = parse_one_hop_entity(EntityId2Index,EntityIndex2Id,news_entity_index,graph,news_index,max_entity_num)

In [ ]:
train_dir = ''    
dev_dir = ''

In [ ]:
train_session = read_train_clickhistory(news_index, data_root_path, train_dir)
train_user = parse_user(news_index, train_session)
train_sess, train_user_id, train_label = get_train_input(news_index, train_session)

In [ ]:
news_fetcher = NewsFetcher(news_title, news_content, news_vert, news_subvert, news_entity)  
test_session = read_test_clickhistory_noclk(news_index, data_root_path, dev_dir)

In [ ]:
test_user = parse_user(news_index, test_session)
test_docids, test_userids, test_labels, test_bound = get_test_input(news_index, test_session)

In [ ]:
batch_size1 = 
batch_size2 = 
epoch_num = 

In [ ]:
train_generator = get_hir_train_generator(news_fetcher, news_entity_index, one_hop_entity, entity_embedding,
                                          train_user['click'], train_user_id, train_sess, train_label, batch_size1)

test_generator = get_test_generator(test_docids,test_userids,news_fetcher,news_entity_index,one_hop_entity,entity_embedding,test_user['click'],batch_size2)

In [ ]:
model, inter_model = create_model_new(title_word_embedding_matrix, content_word_embedding_matrix, entity_dict, category_dict, subcategory_dict)

In [ ]:
model.fit_generator(train_generator, epochs=epoch_num, verbose=1)